In [ ]:
# !pip uninstall tidyzoning -y
# !pip install git+https://github.com/HOUPU1993/tidyzoning.git

Found existing installation: tidyzoning 0.1.0
Uninstalling tidyzoning-0.1.0:
  Successfully uninstalled tidyzoning-0.1.0
  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-i42397vz
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit 67ca8d369981579bd4938e45863ef9c7a0f652e2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tidyzoning: filename=tidyzoning-0.1.0-py3-none-any.whl size=2646 sha256=5585ec8db16b21b423a7c956eb6756a1aab27578ff7cd4c69d08cf7e2b8064c3
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-9vx46x76\wheels\f4\29\51\e431eb95e0bf6589310375ce483daada9c916a6f65f3b65a8f
Successfully built tidyzoning


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-i42397vz'


In [1]:
from tidyzoning import custom_formula

In [4]:
# made a test
print(custom_formula(2, 5))  # x=2, constant=5
print(custom_formula(0, 1))  # x=0, constant=1
print(custom_formula(-3, 8)) # x=-3, constant=8

5
-1
-8
